# 2024 COMP90042 Project

# Readme

#### **BERT Multitask Pretraining**:

In this notebook, we pre-train out custom BERT model jointly on the three tasks: (i) Masked Language Modeling (ii) Sentence entailment (iii) Claim Classification 

We use claims paired with gold passage as positive entailment samples and claims paired with randomly chosen evidences from the knowledge source as negatvie samples.


*** **PLEASE NOTE**: We import helper functions that we implemented for pre-processing/cleaning our data from the python script called `utils.py`. Our ***custom BERT model implementation*** is contained in the python script called `min_bert_multi.py`. And, our **custom WordPiece Tokenizer** implementation is contained is the python script `wordpiece_tokenizer.py`.

In [1]:
%load_ext autoreload
%autoreload 2

# install required packages
!pip install unidecode
!python -m nltk.downloader stopwords
!pip install wandb

from utils import *
from wordpiece_tokenizer import *
from min_bert_multi import *

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn

from tqdm import tqdm
import pickle 
import wandb
import psutil
import random

#wandb.login()
print(torch.cuda.is_available())

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tanzids. Use `wandb login --relogin` to force relogin


True


# 1.DataSet Processing

#### Load the Claims Dataset with Knowledge Source and Clean the Text

In [2]:
# load dataset and prepare corpus
knowledge_source, train_data, val_data = load_dataset()      
print(f"Number of evidence passages: {len(knowledge_source)}")
print(f"Number of training instances: {len(train_data)}")  
print(f"Number of validation instances: {len(val_data)}")

# clean all senteneces in the dataset (this involves converting from unicode to asc-ii, removing URLS, removing repeating non-alphanumeric characters, etc. Just a bunch of thing that are most likely will not be useful for claim classification task)
cleaner = SentenceCleaner()
knowledge_source, train_data, val_data = cleaner.clean_dataset(knowledge_source, train_data, val_data)
print(f"\nNumber of evidence passages after cleaning: {len(knowledge_source)}")
print(f"Number of training instances after cleaning: {len(train_data)}")  
print(f"Number of validation instances after cleaning: {len(val_data)}")

# dictionary for mapping integer to document id 
int2docID = {i:evidence_id for i,evidence_id in enumerate(list(knowledge_source.keys()))}

claim_ids = [claim_id for claim_id in train_data.keys()]

# load trained wordpiece tokenizer from file
with open('tokenizer_worpiece_20000_aug.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# load hard negatives from file if available
#with open("hard_negatives_2.pkl", "rb") as file:
#    hard_negatives = pickle.load(file)  
hard_negatives = None


Number of evidence passages: 1208827
Number of training instances: 1228
Number of validation instances: 154

Number of evidence passages after cleaning: 1206800
Number of training instances after cleaning: 1228
Number of validation instances after cleaning: 154


#### Set up the Multi-Task Pre-Training Dataset.

We have two different types of datasets. In the first dataset `BERTDataset`, each instance is a claim paired with a single evidence passage. In the second dataset `BERTDataset_Long`, each claim is paired with a concatenation of multiple evidence passages.

In [3]:
# pre-training dataset 1
class BERTDataset(Dataset):
    def __init__(self, claims_data, document_store, tokenizer, block_size, hard_negatives=None, mlm_prob=0.20, mask_fraction=0.8, span_mask=False, max_span_len=2):
        self.claims_data = claims_data          # corpus sentences
        self.document_store = document_store    # document store
        self.hard_negatives = hard_negatives    # hard negatives
        self.tokenizer = tokenizer    # wordpiece tokenizer
        self.block_size = block_size  # truncation/max length of sentences
        self.mlm_prob = mlm_prob
        self.mask_fraction = mask_fraction
        self.vocab_size = tokenizer.vocab_size()
        self.span_mask = span_mask
        self.max_span_len = max_span_len
        self.negative_label = 0
        self.positive_label = 1
        self.claim_label2int = {'SUPPORTS':0, 'REFUTES':1, 'NOT_ENOUGH_INFO':2, 'DISPUTED':3}
        self.document_ids = list(document_store.keys())
        self.sent_pairs = self.create_pairs()        
        

    # create positive and negative sentence entailment pairs    
    def create_pairs(self):
        sent_pairs = []
        for claim_id in self.claims_data.keys():
            # get claim label
            claim_label = self.claim_label2int[self.claims_data[claim_id]['claim_label']]
            # shuffle the gold evidences list
            gold_evidences = self.claims_data[claim_id]['evidences']
            random.shuffle(gold_evidences)
            # create a positive entailment pair between each claim and its gold evidences evidence and each possible pair of gold evidences
            for i, evidence_id_1 in enumerate(gold_evidences):
                # randomize the order of claim and evidence in the pair
                if random.random() > 0.5:
                    sent_pairs.append((claim_id, evidence_id_1, self.positive_label, claim_label))  
                else:
                    sent_pairs.append((evidence_id_1, claim_id, self.positive_label, claim_label))
                for evidence_id_2 in gold_evidences[i+1:]:
                    if random.random() > 0.5:
                        sent_pairs.append((evidence_id_1, evidence_id_2, self.positive_label, claim_label))
                    else:
                        sent_pairs.append((evidence_id_2, evidence_id_1, self.positive_label, claim_label))
                
                # create a negative pair with the claim and a randomly chosen document, assign NOT_ENOUGH_INFO claim label to negative pairs
                negative_ev_id = random.choice(self.document_ids)
                if random.random() > 0.5:
                    sent_pairs.append((claim_id, negative_ev_id, self.negative_label, self.claim_label2int['NOT_ENOUGH_INFO']))  
                else:
                    sent_pairs.append((negative_ev_id, claim_id, self.negative_label, self.claim_label2int['NOT_ENOUGH_INFO']))
                
                # also sample a hard negative
                if self.hard_negatives is not None:
                    hard_negative_id = random.choice(self.hard_negatives[claim_id])
                    if random.random() > 0.5:
                        sent_pairs.append((claim_id, hard_negative_id, self.negative_label, self.claim_label2int['NOT_ENOUGH_INFO']))  
                    else:
                        sent_pairs.append((hard_negative_id, claim_id, self.negative_label, self.claim_label2int['NOT_ENOUGH_INFO']))

                # create a negative pair with each gold evidence with a randomly chosen document
                for evidence_id in gold_evidences:
                    negative_ev_id = random.choice(self.document_ids)
                    if random.random() > 0.5:
                        sent_pairs.append((evidence_id, negative_ev_id, self.negative_label, self.claim_label2int['NOT_ENOUGH_INFO']))  
                    else:
                        sent_pairs.append((negative_ev_id, evidence_id, self.negative_label, self.claim_label2int['NOT_ENOUGH_INFO']))

                # create a negative pair with two random documents
                negative_ev_id_1 = random.choice(self.document_ids)
                negative_ev_id_2 = random.choice(self.document_ids)
                sent_pairs.append((negative_ev_id_1, negative_ev_id_2, self.negative_label, self.claim_label2int['NOT_ENOUGH_INFO']))        

        # shuffle the sentence pairs 
        random.shuffle(sent_pairs)                
        return sent_pairs
    

    def __len__(self):
        return len(self.sent_pairs)

    def get_pair_text(self, idx):
         # get sentence ids and entailment label
        sent_1_id, sent_2_id, entailment_label, claim_label = self.sent_pairs[idx]
        # get the actual sentences based on ids
        sent_1_type = sent_1_id.split("-")[0]
        sent_2_type = sent_2_id.split("-")[0]
        if sent_1_type == "claim":
            sent_1 = self.claims_data[sent_1_id]['claim_text']
        else:
            sent_1 = self.document_store[sent_1_id]
        if sent_2_type == "claim":
            sent_2 = self.claims_data[sent_2_id]['claim_text']
        else:
            sent_2 = self.document_store[sent_2_id]            
        return sent_1, sent_2, entailment_label, claim_label


    def __getitem__(self, idx):
        # get sentence ids and entailment label
        sent_1_id, sent_2_id, entailment_label, claim_label = self.sent_pairs[idx]
        # get the actual sentences based on ids
        sent_1_type = sent_1_id.split("-")[0]
        sent_2_type = sent_2_id.split("-")[0]
        if sent_1_type == "claim":
            sent_1 = self.claims_data[sent_1_id]['claim_text']
        else:
            sent_1 = self.document_store[sent_1_id]
        if sent_2_type == "claim":
            sent_2 = self.claims_data[sent_2_id]['claim_text']
        else:
            sent_2 = self.document_store[sent_2_id]            

        # encode both sentence using the tokenizer
        s1_idx = self.tokenizer.encode([sent_1])[0]
        s2_idx = self.tokenizer.encode([sent_2])[0]

        # check if combined length is within block_size-2
        if len(s1_idx) + len(s2_idx) + 3 > self.block_size:
            # calculate the space available for each sentence
            available_space = (self.block_size - 3) // 2
            if len(s1_idx) < available_space:
                # if s1 is shorter than available space, allocate the remaining space to s2
                available_space_s2 = self.block_size - 3 - len(s1_idx)
                if len(s2_idx) > available_space_s2:
                    # if s2 is longer than the available space, crop it
                    start = random.randint(0, len(s2_idx) - available_space_s2)
                    s2_idx = s2_idx[start:start+available_space_s2]
            elif len(s2_idx) < available_space:
                # if s2 is shorter than available space, allocate the remaining space to s1
                available_space_s1 = self.block_size - 3 - len(s2_idx)
                if len(s1_idx) > available_space_s1:
                    # if s1 is longer than the available space, crop it
                    start = random.randint(0, len(s1_idx) - available_space_s1)
                    s1_idx = s1_idx[start:start+available_space_s1]
            else:
                # if both sentences are longer than available space, crop both
                start = random.randint(0, len(s1_idx) - available_space)
                s1_idx = s1_idx[start:start+available_space]
                start = random.randint(0, len(s2_idx) - available_space)
                s2_idx = s2_idx[start:start+available_space]

        # apply masking
        if self.span_mask:
            s1_idx, MLM_label_s1 = self.replace_tokens_span(s1_idx)
            s2_idx, MLM_label_s2 = self.replace_tokens_span(s2_idx)
        else:
            s1_idx, MLM_label_s1 = self.replace_tokens(s1_idx)
            s2_idx, MLM_label_s2 = self.replace_tokens(s2_idx)


        # combine the two sentences with a separator token into single sequence        
        s = torch.cat([torch.tensor([self.tokenizer.cls_token_id()]), s1_idx, torch.tensor([self.tokenizer.sep_token_id()]), s2_idx, torch.tensor([self.tokenizer.sep_token_id()])])
        MLM_label = torch.cat([torch.tensor([-100]), MLM_label_s1, torch.tensor([-100]), MLM_label_s2, torch.tensor([-100])])
        # apply padding
        pad_len = max(0,self.block_size-len(s))
        s = torch.cat([s, torch.full((pad_len,), self.tokenizer.pad_token_id())])
        MLM_label = torch.cat([MLM_label,  torch.full((pad_len,),-100)])    
        attention_mask = torch.cat([torch.ones(self.block_size-pad_len), torch.zeros(pad_len)])
        entailment_label = torch.tensor(entailment_label)
        claim_label = torch.tensor(claim_label)

        # create segment ids
        segment_ids = torch.zeros(self.block_size)
        sep_idx = (s == self.tokenizer.sep_token_id()).nonzero(as_tuple=False)
        segment_ids[sep_idx[0]+1:] = 1
        segment_ids = segment_ids.long()

        return {"masked_input" : s, "MLM_label" : MLM_label, "entailment_label" : entailment_label, "claim_label": claim_label, "attention_mask" : attention_mask, "segment_ids" : segment_ids}


    def on_epoch_end(self):
        self.sent_pairs = self.create_pairs()

    # function for encoding a custom out of corpus sentence
    def encode_custom(self, sent_1, sent_2):
       # encode both sentence using the tokenizer
        s1_idx = self.tokenizer.encode([sent_1])[0]
        s2_idx = self.tokenizer.encode([sent_2])[0]

        # check if combined length is within block_size-2
        if len(s1_idx) + len(s2_idx) + 3 > self.block_size:
            # calculate the space available for each sentence
            available_space = (self.block_size - 3) // 2
            if len(s1_idx) < available_space:
                # if s1 is shorter than available space, allocate the remaining space to s2
                available_space_s2 = self.block_size - 3 - len(s1_idx)
                if len(s2_idx) > available_space_s2:
                    # if s2 is longer than the available space, crop it
                    start = random.randint(0, len(s2_idx) - available_space_s2)
                    s2_idx = s2_idx[start:start+available_space_s2]
            elif len(s2_idx) < available_space:
                # if s2 is shorter than available space, allocate the remaining space to s1
                available_space_s1 = self.block_size - 3 - len(s2_idx)
                if len(s1_idx) > available_space_s1:
                    # if s1 is longer than the available space, crop it
                    start = random.randint(0, len(s1_idx) - available_space_s1)
                    s1_idx = s1_idx[start:start+available_space_s1]
            else:
                # if both sentences are longer than available space, crop both
                start = random.randint(0, len(s1_idx) - available_space)
                s1_idx = s1_idx[start:start+available_space]
                start = random.randint(0, len(s2_idx) - available_space)
                s2_idx = s2_idx[start:start+available_space]

        # apply masking
        if self.span_mask:
            s1_idx, MLM_label_s1 = self.replace_tokens_span(s1_idx)
            s2_idx, MLM_label_s2 = self.replace_tokens_span(s2_idx)
        else:
            s1_idx, MLM_label_s1 = self.replace_tokens(s1_idx)
            s2_idx, MLM_label_s2 = self.replace_tokens(s2_idx)


        # combine the two sentences with a separator token into single sequence        
        s = torch.cat([torch.tensor([self.tokenizer.cls_token_id()]), s1_idx, torch.tensor([self.tokenizer.sep_token_id()]), s2_idx, torch.tensor([self.tokenizer.sep_token_id()])])
        MLM_label = torch.cat([torch.tensor([-100]), MLM_label_s1, torch.tensor([-100]), MLM_label_s2, torch.tensor([-100])])
        # apply padding
        pad_len = max(0,self.block_size-len(s))
        s = torch.cat([s, torch.full((pad_len,), self.tokenizer.pad_token_id())])
        MLM_label = torch.cat([MLM_label,  torch.full((pad_len,),-100)])    
        attention_mask = torch.cat([torch.ones(self.block_size-pad_len), torch.zeros(pad_len)])

        # create segment ids
        segment_ids = torch.zeros(self.block_size)
        sep_idx = (s == self.tokenizer.sep_token_id()).nonzero(as_tuple=False)
        segment_ids[sep_idx[0]+1:] = 1
        segment_ids = segment_ids.long()

        return {"masked_input" : s, "MLM_label" : MLM_label, "attention_mask" : attention_mask, "segment_ids" : segment_ids}

    # randomly replace tokens with mlm_prob probability
    def replace_tokens(self, s):
        # the labels for a masked token is the original token index and -100 for non-masked tokens
        s = torch.tensor(s, dtype=torch.long)
        label = torch.full_like(s, -100)
        # select tokens to be masked with mlm_prob probability
        mask = torch.rand(len(s)) < self.mlm_prob # mask for replacing tokens
        selected_idx = mask.nonzero(as_tuple=False).flatten()
        
        # mask_fraction percenttage of the selected tokens are replaced with mask tokens
        num_masked = int(self.mask_fraction * len(selected_idx))
        # half of the remaining selected tokens are replaced with random tokens, remaining half will be untouched
        num_replaced = int(0.5*(len(selected_idx) - num_masked))

        # shuffle the selcted indices
        shuffled_idx = selected_idx[torch.randperm(len(selected_idx))]
        # make a copy of original tokens before masking
        s_original = s[shuffled_idx].clone()
        # replace with mask tokens
        mask_idx = shuffled_idx[:num_masked]
        s[mask_idx] = self.tokenizer.mask_token_id()
        # replace with random tokens
        replace_idx = shuffled_idx[num_masked:num_masked+num_replaced]
        s[replace_idx] = torch.randint(self.vocab_size, (num_replaced,))
        
        # set labels
        label[shuffled_idx] = s_original
        return s, label
    
    # span masking
    def replace_tokens_span(self, s):
        # the labels for a masked token is the original token index and -100 for non-masked tokens
        s = torch.tensor(s, dtype=torch.long)
        label = torch.full_like(s, -100)
        # determine the number of spans to mask, ensuring at least one span is selected
        avg_span_length = (1+self.max_span_len)/2 # expected span length
        num_spans = max(1, int(self.mlm_prob * len(s) / avg_span_length))
        # generate all possible start positions
        all_starts = torch.arange(0, len(s))
        # shuffle the start positions
        perm = torch.randperm(all_starts.nelement())
        all_starts = all_starts.view(-1)[perm].view(all_starts.size())
        # select the first num_spans positions
        span_starts = all_starts[:num_spans]
        # sort the selected positions
        span_starts, _ = torch.sort(span_starts)

        for i, start in enumerate(span_starts):
            # select a random span length between 1 and 3
            span_length = torch.randint(1, self.max_span_len+1, (1,)).item()
            end = start + span_length
            # if there's a next start position and the current end position is greater, adjust the span length
            if i < num_spans - 1 and end > span_starts[i+1]:
                end = span_starts[i+1]
                span_length = end - start
            # if the number of tokens is less than the span length, adjust the span length
            if end > len(s):
                end = len(s)
                span_length = end - start
            # make a copy of original tokens before masking
            s_original = s[start:end].clone()
            # generate a random number to decide whether to mask or replace the span
            rand_num = torch.rand(1).item()
            if rand_num < self.mask_fraction:
                # replace with mask tokens
                s[start:end] = self.tokenizer.mask_token_id()
            else:
                # replace with random tokens
                s[start:end] = torch.randint(self.vocab_size, (span_length,))
            # set labels
            label[start:end] = s_original

        return s, label


# pre-training dataset 2
class BERTDataset_Long(Dataset):
    def __init__(self, claims_data, document_store, tokenizer, block_size, hard_negatives=None, mlm_prob=0.20, mask_fraction=0.8, span_mask=False, max_span_len=2):
        self.claims_data = claims_data          # corpus sentences
        self.document_store = document_store    # document store
        self.hard_negatives = hard_negatives    # hard negatives
        self.tokenizer = tokenizer    # wordpiece tokenizer
        self.block_size = block_size  # truncation/max length of sentences
        self.mlm_prob = mlm_prob
        self.mask_fraction = mask_fraction
        self.vocab_size = tokenizer.vocab_size()
        self.span_mask = span_mask
        self.max_span_len = max_span_len
        self.negative_label = 0
        self.positive_label = 1
        self.claim_label2int = {'SUPPORTS':0, 'REFUTES':1, 'NOT_ENOUGH_INFO':2, 'DISPUTED':3}
        self.document_ids = list(document_store.keys())
        self.sent_pairs = self.create_pairs()        
        

    # create positive and negative sentence entailment pairs    
    def create_pairs(self):
        sent_pairs = []
        for claim_id in self.claims_data.keys():
            # get claim label
            claim_label = self.claim_label2int[self.claims_data[claim_id]['claim_label']]
            gold_evidences = self.claims_data[claim_id]['evidences']
            
            # create positive example with claim and all its gold evidences
            sent_pairs.append(([claim_id],gold_evidences, self.positive_label, claim_label))
            
            # create a positive example with only gold evidences if there are more than 1
            if len(gold_evidences) > 1:
                sent_pairs.append((gold_evidences[0:1], gold_evidences[1:], self.positive_label, claim_label))
            
            # create a negative example with claim and 3 hard negatives
            if self.hard_negatives is not None:
                hard_negatives = random.sample(self.hard_negatives[claim_id], 3)
                sent_pairs.append(([claim_id], hard_negatives, self.negative_label, self.claim_label2int['NOT_ENOUGH_INFO']))
            
            # create a negative example with each gold evidence and 3 random documents
            for evidence_id in gold_evidences:
                negs = random.sample(self.document_ids, 3)
                sent_pairs.append(([evidence_id], negs, self.negative_label, self.claim_label2int['NOT_ENOUGH_INFO']))

            # create a negative example with 4 random documents
            negs = random.sample(self.document_ids, 4)
            sent_pairs.append((negs[0:1], negs[1:], self.negative_label, self.claim_label2int['NOT_ENOUGH_INFO']))

        # shuffle the sentence pairs 
        random.shuffle(sent_pairs)                
        return sent_pairs
    

    def __len__(self):
        return len(self.sent_pairs)

    def get_pair_text(self, idx):
        # get sentence ids and entailment label
        sent_1_id, sent_2_ids, entailment_label, claim_label = self.sent_pairs[idx]

        # get segment 1 text
        sent_1_id = sent_1_id[0]    
        sent_1_type = sent_1_id.split("-")[0]
        if sent_1_type == "claim":
            sent_1_text = self.claims_data[sent_1_id]['claim_text']
        else:
            sent_1_text = self.document_store[sent_1_id]

        # get segment 2 text
        sent_2_texts = [self.document_store[sent_2_id] for sent_2_id in sent_2_ids]
        sent_2_text = " ".join(sent_2_texts)      
       
        return sent_1_text, sent_2_text, entailment_label, claim_label


    def __getitem__(self, idx):
        # get sentence ids and entailment label
        sent_1_id, sent_2_ids, entailment_label, claim_label = self.sent_pairs[idx]

        # get segment 1 text
        sent_1_id = sent_1_id[0]    
        sent_1_type = sent_1_id.split("-")[0]
        if sent_1_type == "claim":
            sent_1_text = self.claims_data[sent_1_id]['claim_text']
        else:
            sent_1_text = self.document_store[sent_1_id]

        # get segment 2 text
        sent_2_texts = [self.document_store[sent_2_id] for sent_2_id in sent_2_ids]
        sent_2_text = " ".join(sent_2_texts)   

        # encode both sentence using the tokenizer
        s1_idx = self.tokenizer.encode([sent_1_text])[0]
        s2_idx = self.tokenizer.encode([sent_2_text])[0]

        # check if combined length is within block_size-2
        if len(s1_idx) + len(s2_idx) + 3 > self.block_size:
            # calculate the space available for each sentence
            available_space = (self.block_size - 3) // 2
            if len(s1_idx) < available_space:
                # if s1 is shorter than available space, allocate the remaining space to s2
                available_space_s2 = self.block_size - 3 - len(s1_idx)
                if len(s2_idx) > available_space_s2:
                    # if s2 is longer than the available space, crop it
                    start = random.randint(0, len(s2_idx) - available_space_s2)
                    s2_idx = s2_idx[start:start+available_space_s2]
            elif len(s2_idx) < available_space:
                # if s2 is shorter than available space, allocate the remaining space to s1
                available_space_s1 = self.block_size - 3 - len(s2_idx)
                if len(s1_idx) > available_space_s1:
                    # if s1 is longer than the available space, crop it
                    start = random.randint(0, len(s1_idx) - available_space_s1)
                    s1_idx = s1_idx[start:start+available_space_s1]
            else:
                # if both sentences are longer than available space, crop both
                start = random.randint(0, len(s1_idx) - available_space)
                s1_idx = s1_idx[start:start+available_space]
                start = random.randint(0, len(s2_idx) - available_space)
                s2_idx = s2_idx[start:start+available_space]

        # apply masking
        if self.span_mask:
            s1_idx, MLM_label_s1 = self.replace_tokens_span(s1_idx)
            s2_idx, MLM_label_s2 = self.replace_tokens_span(s2_idx)
        else:
            s1_idx, MLM_label_s1 = self.replace_tokens(s1_idx)
            s2_idx, MLM_label_s2 = self.replace_tokens(s2_idx)


        # combine the two sentences with a separator token into single sequence        
        s = torch.cat([torch.tensor([self.tokenizer.cls_token_id()]), s1_idx, torch.tensor([self.tokenizer.sep_token_id()]), s2_idx, torch.tensor([self.tokenizer.sep_token_id()])])
        MLM_label = torch.cat([torch.tensor([-100]), MLM_label_s1, torch.tensor([-100]), MLM_label_s2, torch.tensor([-100])])
        # apply padding
        pad_len = max(0,self.block_size-len(s))
        s = torch.cat([s, torch.full((pad_len,), self.tokenizer.pad_token_id())])
        MLM_label = torch.cat([MLM_label,  torch.full((pad_len,),-100)])    
        attention_mask = torch.cat([torch.ones(self.block_size-pad_len), torch.zeros(pad_len)])
        entailment_label = torch.tensor(entailment_label)
        claim_label = torch.tensor(claim_label)

        # create segment ids
        segment_ids = torch.zeros(self.block_size)
        sep_idx = (s == self.tokenizer.sep_token_id()).nonzero(as_tuple=False)
        segment_ids[sep_idx[0]+1:] = 1
        segment_ids = segment_ids.long()

        return {"masked_input" : s, "MLM_label" : MLM_label, "entailment_label" : entailment_label, "claim_label": claim_label, "attention_mask" : attention_mask, "segment_ids" : segment_ids}

    def on_epoch_end(self):
        self.sent_pairs = self.create_pairs()

    # function for encoding a custom out of corpus sentence
    def encode_custom(self, sent_1, sent_2):
       # encode both sentence using the tokenizer
        s1_idx = self.tokenizer.encode([sent_1])[0]
        s2_idx = self.tokenizer.encode([sent_2])[0]

        # check if combined length is within block_size-2
        if len(s1_idx) + len(s2_idx) + 3 > self.block_size:
            # calculate the space available for each sentence
            available_space = (self.block_size - 3) // 2
            if len(s1_idx) < available_space:
                # if s1 is shorter than available space, allocate the remaining space to s2
                available_space_s2 = self.block_size - 3 - len(s1_idx)
                if len(s2_idx) > available_space_s2:
                    # if s2 is longer than the available space, crop it
                    start = random.randint(0, len(s2_idx) - available_space_s2)
                    s2_idx = s2_idx[start:start+available_space_s2]
            elif len(s2_idx) < available_space:
                # if s2 is shorter than available space, allocate the remaining space to s1
                available_space_s1 = self.block_size - 3 - len(s2_idx)
                if len(s1_idx) > available_space_s1:
                    # if s1 is longer than the available space, crop it
                    start = random.randint(0, len(s1_idx) - available_space_s1)
                    s1_idx = s1_idx[start:start+available_space_s1]
            else:
                # if both sentences are longer than available space, crop both
                start = random.randint(0, len(s1_idx) - available_space)
                s1_idx = s1_idx[start:start+available_space]
                start = random.randint(0, len(s2_idx) - available_space)
                s2_idx = s2_idx[start:start+available_space]

        # apply masking
        if self.span_mask:
            s1_idx, MLM_label_s1 = self.replace_tokens_span(s1_idx)
            s2_idx, MLM_label_s2 = self.replace_tokens_span(s2_idx)
        else:
            s1_idx, MLM_label_s1 = self.replace_tokens(s1_idx)
            s2_idx, MLM_label_s2 = self.replace_tokens(s2_idx)


        # combine the two sentences with a separator token into single sequence        
        s = torch.cat([torch.tensor([self.tokenizer.cls_token_id()]), s1_idx, torch.tensor([self.tokenizer.sep_token_id()]), s2_idx, torch.tensor([self.tokenizer.sep_token_id()])])
        MLM_label = torch.cat([torch.tensor([-100]), MLM_label_s1, torch.tensor([-100]), MLM_label_s2, torch.tensor([-100])])
        # apply padding
        pad_len = max(0,self.block_size-len(s))
        s = torch.cat([s, torch.full((pad_len,), self.tokenizer.pad_token_id())])
        MLM_label = torch.cat([MLM_label,  torch.full((pad_len,),-100)])    
        attention_mask = torch.cat([torch.ones(self.block_size-pad_len), torch.zeros(pad_len)])

        # create segment ids
        segment_ids = torch.zeros(self.block_size)
        sep_idx = (s == self.tokenizer.sep_token_id()).nonzero(as_tuple=False)
        segment_ids[sep_idx[0]+1:] = 1
        segment_ids = segment_ids.long()

        return {"masked_input" : s, "MLM_label" : MLM_label, "attention_mask" : attention_mask, "segment_ids" : segment_ids}

    # randomly replace tokens with mlm_prob probability
    def replace_tokens(self, s):
        # the labels for a masked token is the original token index and -100 for non-masked tokens
        s = torch.tensor(s, dtype=torch.long)
        label = torch.full_like(s, -100)
        # select tokens to be masked with mlm_prob probability
        mask = torch.rand(len(s)) < self.mlm_prob # mask for replacing tokens
        selected_idx = mask.nonzero(as_tuple=False).flatten()
        
        # mask_fraction percenttage of the selected tokens are replaced with mask tokens
        num_masked = int(self.mask_fraction * len(selected_idx))
        # half of the remaining selected tokens are replaced with random tokens, remaining half will be untouched
        num_replaced = int(0.5*(len(selected_idx) - num_masked))

        # shuffle the selcted indices
        shuffled_idx = selected_idx[torch.randperm(len(selected_idx))]
        # make a copy of original tokens before masking
        s_original = s[shuffled_idx].clone()
        # replace with mask tokens
        mask_idx = shuffled_idx[:num_masked]
        s[mask_idx] = self.tokenizer.mask_token_id()
        # replace with random tokens
        replace_idx = shuffled_idx[num_masked:num_masked+num_replaced]
        s[replace_idx] = torch.randint(self.vocab_size, (num_replaced,))
        
        # set labels
        label[shuffled_idx] = s_original
        return s, label
    
    # span masking
    def replace_tokens_span(self, s):
        # the labels for a masked token is the original token index and -100 for non-masked tokens
        s = torch.tensor(s, dtype=torch.long)
        label = torch.full_like(s, -100)
        # determine the number of spans to mask, ensuring at least one span is selected
        avg_span_length = (1+self.max_span_len)/2 # expected span length
        num_spans = max(1, int(self.mlm_prob * len(s) / avg_span_length))
        # generate all possible start positions
        all_starts = torch.arange(0, len(s))
        # shuffle the start positions
        perm = torch.randperm(all_starts.nelement())
        all_starts = all_starts.view(-1)[perm].view(all_starts.size())
        # select the first num_spans positions
        span_starts = all_starts[:num_spans]
        # sort the selected positions
        span_starts, _ = torch.sort(span_starts)

        for i, start in enumerate(span_starts):
            # select a random span length between 1 and 3
            span_length = torch.randint(1, self.max_span_len+1, (1,)).item()
            end = start + span_length
            # if there's a next start position and the current end position is greater, adjust the span length
            if i < num_spans - 1 and end > span_starts[i+1]:
                end = span_starts[i+1]
                span_length = end - start
            # if the number of tokens is less than the span length, adjust the span length
            if end > len(s):
                end = len(s)
                span_length = end - start
            # make a copy of original tokens before masking
            s_original = s[start:end].clone()
            # generate a random number to decide whether to mask or replace the span
            rand_num = torch.rand(1).item()
            if rand_num < self.mask_fraction:
                # replace with mask tokens
                s[start:end] = self.tokenizer.mask_token_id()
            else:
                # replace with random tokens
                s[start:end] = torch.randint(self.vocab_size, (span_length,))
            # set labels
            label[start:end] = s_original

        return s, label


# 2. Model Implementation

#### Prepare the Training and Validation Dataloaders

In [4]:
# create dataset
block_size = 128
batch_size = 48

train_dataset = BERTDataset_Long(train_data, knowledge_source, tokenizer, block_size,  hard_negatives=hard_negatives, mlm_prob=0.15, span_mask=False)
val_dataset = BERTDataset_Long(val_data, knowledge_source, tokenizer, block_size, mlm_prob=0.15, span_mask=False)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=2)  # set pin_memory for faster pre-fetching
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=1)  # set pin_memory for faster pre-fetching 
print(f"Total number of training batches: {len(train_dataloader)}")
print(f"Total number of validation batches: {len(val_dataloader)}")

Total number of training batches: 184
Total number of validation batches: 20


#### Instantiate the custom BERT model, set the hyperparameter values

In [7]:
# model hyperparameters
embedding_dim = 512
head_size = embedding_dim
num_heads = 16
num_layers = 8
dropout_rate = 0.1
num_epochs = 5
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu' 

model = BERTModel(vocab_size=tokenizer.vocab_size(), block_size=block_size, embedding_dim=embedding_dim, head_size=head_size, num_heads=num_heads, num_layers=num_layers, pad_token_id=tokenizer.pad_token_id(), device=device)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# load from previous checkpoint
#model, optimizer, epoch, loss = load_bert_model_checkpoint(model, optimizer, name="BERT_multitask_checkpoint_entaiment_claims_long_600_epochs", device=device, strict=False)

# learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=1e-6)

num_params = sum(p.numel() for p in model.parameters())
print(f"Device: {device}")
print(f"Total number of parameters in transformer network: {num_params/1e6} M")
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

/home/tanzid/miniconda3/envs/torch_clone_2/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Loaded pretrained BERT model checkpoint at epoch 20 with loss 2.0593682737607804
Device: cuda
Total number of parameters in transformer network: 45.030221 M
RAM used: 3426.89 MB


In [8]:
"""
run = wandb.init(
    project="BERT Pretrain MLM", 
    config={
        "model": "BERT-like Transformer Encoder",
        "learning_rate": learning_rate, 
        "epochs": num_epochs,
        "batch_size": batch_size, 
        "corpus": "Climate Claims"},)   

def log_metrics(metrics):
    wandb.log(metrics)
"""

#### Start pre-training the model.

In [8]:
train(model, 200, train_dataloader, val_dataloader, optimizer, grad_accumulation_steps=20, val_every=1000, save_every=4, device=device, log_metrics=None, mixed_precision=True, checkpoint_name="BERT_multitask", scheduler=scheduler, include_claim_loss=False)

Epoch 1, Train Loss(Total, MLM, Entailment, Claim): (2.355, 2.199, 0.157, 1.247), Train Accuracy (MLM, Entailment, Claim): (0.603, 0.948, 0.463), Val Loss: 6.003, Val Accuracy (MLM, Entailment, Claim): (0.376, 0.864, 0.514): 100%|██████████| 1238/1238 [03:54<00:00,  5.29it/s]
Epoch 2, Train Loss(Total, MLM, Entailment, Claim): (2.259, 2.124, 0.135, 1.253), Train Accuracy (MLM, Entailment, Claim): (0.606, 0.952, 0.470), Val Loss: 5.963, Val Accuracy (MLM, Entailment, Claim): (0.376, 0.867, 0.594): 100%|██████████| 1238/1238 [04:00<00:00,  5.15it/s]
Epoch 3, Train Loss(Total, MLM, Entailment, Claim): (2.127, 2.028, 0.099, 1.252), Train Accuracy (MLM, Entailment, Claim): (0.608, 0.955, 0.472), Val Loss: 6.017, Val Accuracy (MLM, Entailment, Claim): (0.378, 0.868, 0.555): 100%|██████████| 1238/1238 [03:58<00:00,  5.18it/s]
Epoch 4, Train Loss(Total, MLM, Entailment, Claim): (2.171, 2.076, 0.094, 1.256), Train Accuracy (MLM, Entailment, Claim): (0.610, 0.957, 0.471), Val Loss: 6.032, Val Ac

Saved BERT model checkpoint!


Epoch 5, Train Loss(Total, MLM, Entailment, Claim): (2.190, 2.081, 0.109, 1.240), Train Accuracy (MLM, Entailment, Claim): (0.611, 0.959, 0.472), Val Loss: 6.054, Val Accuracy (MLM, Entailment, Claim): (0.376, 0.858, 0.557): 100%|██████████| 1238/1238 [04:01<00:00,  5.14it/s]
Epoch 6, Train Loss(Total, MLM, Entailment, Claim): (2.231, 2.124, 0.107, 1.272), Train Accuracy (MLM, Entailment, Claim): (0.612, 0.960, 0.472), Val Loss: 6.060, Val Accuracy (MLM, Entailment, Claim): (0.379, 0.858, 0.552): 100%|██████████| 1238/1238 [04:01<00:00,  5.12it/s]
Epoch 7, Train Loss(Total, MLM, Entailment, Claim): (2.152, 2.048, 0.103, 1.245), Train Accuracy (MLM, Entailment, Claim): (0.612, 0.961, 0.472), Val Loss: 6.042, Val Accuracy (MLM, Entailment, Claim): (0.378, 0.864, 0.557): 100%|██████████| 1238/1238 [04:01<00:00,  5.12it/s]
Epoch 8, Train Loss(Total, MLM, Entailment, Claim): (2.162, 2.092, 0.070, 1.222), Train Accuracy (MLM, Entailment, Claim): (0.612, 0.962, 0.474), Val Loss: 6.121, Val Ac

Saved BERT model checkpoint!


Epoch 9, Train Loss(Total, MLM, Entailment, Claim): (2.233, 2.147, 0.086, 1.206), Train Accuracy (MLM, Entailment, Claim): (0.611, 0.962, 0.477), Val Loss: 6.137, Val Accuracy (MLM, Entailment, Claim): (0.371, 0.865, 0.618): 100%|██████████| 1238/1238 [04:01<00:00,  5.12it/s]
Epoch 10, Train Loss(Total, MLM, Entailment, Claim): (2.250, 2.198, 0.052, 1.248), Train Accuracy (MLM, Entailment, Claim): (0.609, 0.963, 0.481), Val Loss: 6.014, Val Accuracy (MLM, Entailment, Claim): (0.368, 0.855, 0.685): 100%|██████████| 1238/1238 [04:04<00:00,  5.07it/s]
Epoch 11, Train Loss(Total, MLM, Entailment, Claim): (2.255, 2.190, 0.065, 1.289), Train Accuracy (MLM, Entailment, Claim): (0.606, 0.964, 0.477), Val Loss: 6.203, Val Accuracy (MLM, Entailment, Claim): (0.376, 0.849, 0.496): 100%|██████████| 1238/1238 [04:03<00:00,  5.09it/s]
Epoch 12, Train Loss(Total, MLM, Entailment, Claim): (2.154, 2.117, 0.037, 1.343), Train Accuracy (MLM, Entailment, Claim): (0.604, 0.965, 0.468), Val Loss: 6.573, Val

...

Saved BERT model checkpoint!


Epoch 193, Train Loss(Total, MLM, Entailment, Claim): (1.332, 1.326, 0.007, 1.387), Train Accuracy (MLM, Entailment, Claim): (0.672, 0.995, 0.331), Val Loss: 6.373, Val Accuracy (MLM, Entailment, Claim): (0.422, 0.871, 0.293): 100%|██████████| 1238/1238 [03:55<00:00,  5.25it/s]
Epoch 194, Train Loss(Total, MLM, Entailment, Claim): (1.237, 1.237, 0.000, 1.373), Train Accuracy (MLM, Entailment, Claim): (0.672, 0.995, 0.331), Val Loss: 6.432, Val Accuracy (MLM, Entailment, Claim): (0.420, 0.872, 0.311): 100%|██████████| 1238/1238 [03:56<00:00,  5.23it/s]
Epoch 195, Train Loss(Total, MLM, Entailment, Claim): (1.305, 1.303, 0.002, 1.357), Train Accuracy (MLM, Entailment, Claim): (0.673, 0.995, 0.331), Val Loss: 6.571, Val Accuracy (MLM, Entailment, Claim): (0.419, 0.870, 0.300): 100%|██████████| 1238/1238 [03:56<00:00,  5.23it/s]
Epoch 196, Train Loss(Total, MLM, Entailment, Claim): (1.215, 1.214, 0.001, 1.379), Train Accuracy (MLM, Entailment, Claim): (0.673, 0.995, 0.331), Val Loss: 6.514

Saved BERT model checkpoint!


Epoch 197, Train Loss(Total, MLM, Entailment, Claim): (1.237, 1.237, 0.000, 1.346), Train Accuracy (MLM, Entailment, Claim): (0.673, 0.995, 0.331), Val Loss: 6.507, Val Accuracy (MLM, Entailment, Claim): (0.424, 0.867, 0.314): 100%|██████████| 1238/1238 [03:56<00:00,  5.23it/s]
Epoch 198, Train Loss(Total, MLM, Entailment, Claim): (1.220, 1.218, 0.001, 1.420), Train Accuracy (MLM, Entailment, Claim): (0.674, 0.995, 0.330), Val Loss: 6.529, Val Accuracy (MLM, Entailment, Claim): (0.416, 0.885, 0.293): 100%|██████████| 1238/1238 [03:56<00:00,  5.24it/s]
Epoch 199, Train Loss(Total, MLM, Entailment, Claim): (1.247, 1.246, 0.002, 1.435), Train Accuracy (MLM, Entailment, Claim): (0.674, 0.995, 0.330), Val Loss: 6.753, Val Accuracy (MLM, Entailment, Claim): (0.418, 0.855, 0.229): 100%|██████████| 1238/1238 [03:56<00:00,  5.24it/s]
Epoch 200, Train Loss(Total, MLM, Entailment, Claim): (1.280, 1.277, 0.002, 1.394), Train Accuracy (MLM, Entailment, Claim): (0.674, 0.995, 0.330), Val Loss: 6.395

Saved BERT model checkpoint!
Training done!


In [10]:
train(model, 150, train_dataloader, val_dataloader, optimizer, grad_accumulation_steps=20, val_every=1000, save_every=4, device=device, log_metrics=log_metrics, mixed_precision=True, checkpoint_name="BERT_multitask", scheduler=scheduler, include_claim_loss=False)

Epoch 1, Train Loss(Total, MLM, Entailment, Claim): (1.376, 1.368, 0.008, 1.306), Train Accuracy (MLM, Entailment, Claim): (0.707, 0.997, 0.288), Val Loss: 6.558, Val Accuracy (MLM, Entailment, Claim): (0.413, 0.872, 0.278): 100%|██████████| 1238/1238 [03:54<00:00,  5.27it/s]
Epoch 2, Train Loss(Total, MLM, Entailment, Claim): (1.319, 1.314, 0.005, 1.350), Train Accuracy (MLM, Entailment, Claim): (0.706, 0.997, 0.297), Val Loss: 6.599, Val Accuracy (MLM, Entailment, Claim): (0.413, 0.855, 0.332): 100%|██████████| 1238/1238 [03:55<00:00,  5.27it/s]
Epoch 3, Train Loss(Total, MLM, Entailment, Claim): (1.305, 1.303, 0.002, 1.397), Train Accuracy (MLM, Entailment, Claim): (0.709, 0.997, 0.294), Val Loss: 6.578, Val Accuracy (MLM, Entailment, Claim): (0.417, 0.871, 0.290): 100%|██████████| 1238/1238 [03:57<00:00,  5.22it/s]
Epoch 4, Train Loss(Total, MLM, Entailment, Claim): (1.266, 1.264, 0.002, 1.353), Train Accuracy (MLM, Entailment, Claim): (0.713, 0.998, 0.297), Val Loss: 6.434, Val Ac

Saved BERT model checkpoint!


Epoch 5, Train Loss(Total, MLM, Entailment, Claim): (1.227, 1.227, 0.001, 1.360), Train Accuracy (MLM, Entailment, Claim): (0.716, 0.998, 0.301), Val Loss: 6.525, Val Accuracy (MLM, Entailment, Claim): (0.427, 0.864, 0.303): 100%|██████████| 1238/1238 [03:56<00:00,  5.23it/s]
Epoch 6, Train Loss(Total, MLM, Entailment, Claim): (1.260, 1.259, 0.001, 1.351), Train Accuracy (MLM, Entailment, Claim): (0.719, 0.998, 0.303), Val Loss: 6.553, Val Accuracy (MLM, Entailment, Claim): (0.423, 0.872, 0.302): 100%|██████████| 1238/1238 [03:56<00:00,  5.24it/s]
Epoch 7, Train Loss(Total, MLM, Entailment, Claim): (1.227, 1.223, 0.003, 1.359), Train Accuracy (MLM, Entailment, Claim): (0.721, 0.998, 0.303), Val Loss: 6.562, Val Accuracy (MLM, Entailment, Claim): (0.420, 0.871, 0.293): 100%|██████████| 1238/1238 [03:55<00:00,  5.25it/s]
Epoch 8, Train Loss(Total, MLM, Entailment, Claim): (1.277, 1.275, 0.003, 1.384), Train Accuracy (MLM, Entailment, Claim): (0.722, 0.999, 0.305), Val Loss: 6.542, Val Ac

Saved BERT model checkpoint!


Epoch 9, Train Loss(Total, MLM, Entailment, Claim): (1.271, 1.269, 0.001, 1.434), Train Accuracy (MLM, Entailment, Claim): (0.723, 0.999, 0.303), Val Loss: 6.404, Val Accuracy (MLM, Entailment, Claim): (0.422, 0.878, 0.317): 100%|██████████| 1238/1238 [03:57<00:00,  5.21it/s]
Epoch 10, Train Loss(Total, MLM, Entailment, Claim): (1.298, 1.291, 0.008, 1.375), Train Accuracy (MLM, Entailment, Claim): (0.722, 0.999, 0.300), Val Loss: 6.570, Val Accuracy (MLM, Entailment, Claim): (0.417, 0.864, 0.295): 100%|██████████| 1238/1238 [03:58<00:00,  5.18it/s]
Epoch 11, Train Loss(Total, MLM, Entailment, Claim): (1.347, 1.323, 0.023, 1.426), Train Accuracy (MLM, Entailment, Claim): (0.721, 0.998, 0.298), Val Loss: 6.765, Val Accuracy (MLM, Entailment, Claim): (0.414, 0.860, 0.209): 100%|██████████| 1238/1238 [03:58<00:00,  5.18it/s]
Epoch 12, Train Loss(Total, MLM, Entailment, Claim): (1.347, 1.342, 0.005, 1.438), Train Accuracy (MLM, Entailment, Claim): (0.720, 0.998, 0.294), Val Loss: 6.706, Val

...

Saved BERT model checkpoint!


Epoch 145, Train Loss(Total, MLM, Entailment, Claim): (0.991, 0.990, 0.000, 1.310), Train Accuracy (MLM, Entailment, Claim): (0.740, 0.999, 0.280), Val Loss: 6.761, Val Accuracy (MLM, Entailment, Claim): (0.432, 0.870, 0.334): 100%|██████████| 1238/1238 [04:01<00:00,  5.13it/s]
Epoch 146, Train Loss(Total, MLM, Entailment, Claim): (0.997, 0.994, 0.002, 1.313), Train Accuracy (MLM, Entailment, Claim): (0.740, 0.999, 0.281), Val Loss: 6.696, Val Accuracy (MLM, Entailment, Claim): (0.430, 0.876, 0.329): 100%|██████████| 1238/1238 [04:01<00:00,  5.13it/s]
Epoch 147, Train Loss(Total, MLM, Entailment, Claim): (0.975, 0.972, 0.003, 1.340), Train Accuracy (MLM, Entailment, Claim): (0.740, 0.999, 0.281), Val Loss: 6.816, Val Accuracy (MLM, Entailment, Claim): (0.430, 0.863, 0.304): 100%|██████████| 1238/1238 [04:01<00:00,  5.13it/s]
Epoch 148, Train Loss(Total, MLM, Entailment, Claim): (1.015, 1.015, 0.001, 1.369), Train Accuracy (MLM, Entailment, Claim): (0.740, 0.999, 0.281), Val Loss: 6.804

Saved BERT model checkpoint!


Epoch 149, Train Loss(Total, MLM, Entailment, Claim): (0.996, 0.995, 0.001, 1.395), Train Accuracy (MLM, Entailment, Claim): (0.740, 0.999, 0.281), Val Loss: 6.865, Val Accuracy (MLM, Entailment, Claim): (0.431, 0.868, 0.340): 100%|██████████| 1238/1238 [04:02<00:00,  5.11it/s]
Epoch 150, Train Loss(Total, MLM, Entailment, Claim): (1.053, 1.051, 0.002, 1.340), Train Accuracy (MLM, Entailment, Claim): (0.740, 0.999, 0.281), Val Loss: 6.682, Val Accuracy (MLM, Entailment, Claim): (0.431, 0.853, 0.385): 100%|██████████| 1238/1238 [04:02<00:00,  5.10it/s]

Training done!


In [9]:
train(model, 40, train_dataloader, val_dataloader, optimizer, grad_accumulation_steps=20, val_every=180, save_every=4, device=device, log_metrics=log_metrics, mixed_precision=True, checkpoint_name="BERT_multitask", scheduler=scheduler, include_claim_loss=True, claim_loss_weight=0.5)

Epoch 1, Train Loss(Total, MLM, Entailment, Claim): (2.106, 2.069, 0.033, 0.041), Train Accuracy (MLM, Entailment, Claim): (0.636, 0.982, 0.988), Val Loss: 4.384, Val Accuracy (MLM, Entailment, Claim): (0.510, 0.870, 0.844): 100%|██████████| 184/184 [01:41<00:00,  1.82it/s]
Epoch 2, Train Loss(Total, MLM, Entailment, Claim): (2.117, 2.091, 0.024, 0.028), Train Accuracy (MLM, Entailment, Claim): (0.636, 0.986, 0.989), Val Loss: 4.405, Val Accuracy (MLM, Entailment, Claim): (0.512, 0.839, 0.835): 100%|██████████| 184/184 [01:39<00:00,  1.85it/s]
Epoch 3, Train Loss(Total, MLM, Entailment, Claim): (2.077, 2.068, 0.011, 0.008), Train Accuracy (MLM, Entailment, Claim): (0.637, 0.988, 0.991), Val Loss: 4.420, Val Accuracy (MLM, Entailment, Claim): (0.521, 0.859, 0.836): 100%|██████████| 184/184 [01:39<00:00,  1.86it/s]
Epoch 4, Train Loss(Total, MLM, Entailment, Claim): (2.133, 2.126, 0.009, 0.004), Train Accuracy (MLM, Entailment, Claim): (0.638, 0.990, 0.993), Val Loss: 4.539, Val Accuracy

Saved BERT model checkpoint!


Epoch 5, Train Loss(Total, MLM, Entailment, Claim): (2.061, 2.052, 0.008, 0.010), Train Accuracy (MLM, Entailment, Claim): (0.639, 0.991, 0.994), Val Loss: 4.677, Val Accuracy (MLM, Entailment, Claim): (0.513, 0.844, 0.843): 100%|██████████| 184/184 [01:37<00:00,  1.89it/s]
Epoch 6, Train Loss(Total, MLM, Entailment, Claim): (2.046, 2.038, 0.012, 0.005), Train Accuracy (MLM, Entailment, Claim): (0.639, 0.992, 0.994), Val Loss: 4.653, Val Accuracy (MLM, Entailment, Claim): (0.514, 0.846, 0.843): 100%|██████████| 184/184 [01:40<00:00,  1.83it/s]
Epoch 7, Train Loss(Total, MLM, Entailment, Claim): (2.049, 2.043, 0.007, 0.005), Train Accuracy (MLM, Entailment, Claim): (0.640, 0.993, 0.995), Val Loss: 4.700, Val Accuracy (MLM, Entailment, Claim): (0.512, 0.850, 0.837): 100%|██████████| 184/184 [01:45<00:00,  1.75it/s]
Epoch 8, Train Loss(Total, MLM, Entailment, Claim): (2.073, 2.061, 0.018, 0.006), Train Accuracy (MLM, Entailment, Claim): (0.640, 0.993, 0.995), Val Loss: 4.749, Val Accuracy

Saved BERT model checkpoint!


Epoch 9, Train Loss(Total, MLM, Entailment, Claim): (2.013, 2.007, 0.005, 0.006), Train Accuracy (MLM, Entailment, Claim): (0.641, 0.994, 0.995), Val Loss: 4.911, Val Accuracy (MLM, Entailment, Claim): (0.515, 0.854, 0.834): 100%|██████████| 184/184 [01:38<00:00,  1.87it/s]
Epoch 10, Train Loss(Total, MLM, Entailment, Claim): (2.013, 2.002, 0.007, 0.014), Train Accuracy (MLM, Entailment, Claim): (0.641, 0.994, 0.996), Val Loss: 5.021, Val Accuracy (MLM, Entailment, Claim): (0.516, 0.848, 0.835): 100%|██████████| 184/184 [01:37<00:00,  1.88it/s]
Epoch 11, Train Loss(Total, MLM, Entailment, Claim): (2.012, 2.004, 0.007, 0.008), Train Accuracy (MLM, Entailment, Claim): (0.642, 0.994, 0.996), Val Loss: 5.098, Val Accuracy (MLM, Entailment, Claim): (0.509, 0.855, 0.836): 100%|██████████| 184/184 [01:43<00:00,  1.78it/s]
Epoch 12, Train Loss(Total, MLM, Entailment, Claim): (2.003, 1.992, 0.012, 0.009), Train Accuracy (MLM, Entailment, Claim): (0.642, 0.995, 0.996), Val Loss: 4.899, Val Accur

...

Saved BERT model checkpoint!


Epoch 33, Train Loss(Total, MLM, Entailment, Claim): (1.867, 1.858, 0.008, 0.010), Train Accuracy (MLM, Entailment, Claim): (0.651, 0.997, 0.997), Val Loss: 5.087, Val Accuracy (MLM, Entailment, Claim): (0.529, 0.860, 0.843): 100%|██████████| 184/184 [01:40<00:00,  1.84it/s]
Epoch 34, Train Loss(Total, MLM, Entailment, Claim): (1.862, 1.859, 0.001, 0.004), Train Accuracy (MLM, Entailment, Claim): (0.652, 0.997, 0.997), Val Loss: 5.004, Val Accuracy (MLM, Entailment, Claim): (0.521, 0.864, 0.845): 100%|██████████| 184/184 [01:39<00:00,  1.85it/s]
Epoch 35, Train Loss(Total, MLM, Entailment, Claim): (1.861, 1.860, 0.001, 0.003), Train Accuracy (MLM, Entailment, Claim): (0.652, 0.997, 0.997), Val Loss: 5.125, Val Accuracy (MLM, Entailment, Claim): (0.527, 0.863, 0.838): 100%|██████████| 184/184 [01:41<00:00,  1.82it/s]
Epoch 36, Train Loss(Total, MLM, Entailment, Claim): (1.831, 1.828, 0.002, 0.004), Train Accuracy (MLM, Entailment, Claim): (0.652, 0.997, 0.997), Val Loss: 5.104, Val Accu

Saved BERT model checkpoint!


Epoch 37, Train Loss(Total, MLM, Entailment, Claim): (1.818, 1.817, 0.001, 0.002), Train Accuracy (MLM, Entailment, Claim): (0.653, 0.997, 0.998), Val Loss: 5.217, Val Accuracy (MLM, Entailment, Claim): (0.522, 0.854, 0.839): 100%|██████████| 184/184 [01:40<00:00,  1.83it/s]
Epoch 38, Train Loss(Total, MLM, Entailment, Claim): (1.835, 1.833, 0.002, 0.001), Train Accuracy (MLM, Entailment, Claim): (0.653, 0.997, 0.998), Val Loss: 5.307, Val Accuracy (MLM, Entailment, Claim): (0.528, 0.852, 0.839): 100%|██████████| 184/184 [01:40<00:00,  1.83it/s]
Epoch 39, Train Loss(Total, MLM, Entailment, Claim): (1.824, 1.823, 0.002, 0.001), Train Accuracy (MLM, Entailment, Claim): (0.653, 0.997, 0.998), Val Loss: 5.020, Val Accuracy (MLM, Entailment, Claim): (0.521, 0.866, 0.838): 100%|██████████| 184/184 [01:42<00:00,  1.79it/s]
Epoch 40, Train Loss(Total, MLM, Entailment, Claim): (1.812, 1.809, 0.002, 0.003), Train Accuracy (MLM, Entailment, Claim): (0.654, 0.997, 0.998), Val Loss: 5.229, Val Accu

Saved BERT model checkpoint!
Training done!


In [ ]:
save_bert_model_checkpoint(model, optimizer, name='BERT_multitask_checkpoint_entaiment_claims_long_600_epochs')

# 3.Testing and Evaluation

#### We now run some quick tests to check the performance of the pretrained language model on each of the three tasks: (i) Masked token prediction, (ii) Sentence Entailment Prediction and (iii) Claim Class Prediction

In [22]:
# get pair of sentences from training dataset
idx = 275
sentence_1, sentence_2, entailment_label, claim_label = val_dataset.get_pair_text(idx)
print(sentence_1)
print(sentence_2)
print(entailment_label, claim_label)

# encode the sentence
input = train_dataset.encode_custom(sentence_1, sentence_2)
encoded_idx = input['masked_input'].tolist()

# convert to subword tokens
sentence_1_subwords = tokenizer.encode([sentence_1], return_subwords=True)[1][0]
sentence_2_subwords = tokenizer.encode([sentence_2], return_subwords=True)[1][0]
combined_sentence_subwords =  ['[CLS]']  + sentence_1_subwords + ['[SEP]'] + sentence_2_subwords + ['[SEP]']
masked_subwords = [tokenizer.int2word[i] for i in encoded_idx]

"Dominance hierarchies, diversity and species richness of vascular plants in an alpine meadow: contrasting short and medium term responses to simulated global change".
In 2017, they held seven Tag Team Championships from seven different promotions at the same time, including the ROH World Tag Team Championship. Local elections are to be held for Wyre Borough Council on 7 May 2015, the same day as the United Kingdom general election, 2015 and other United Kingdom local elections, 2015. It was declared as ` sanctuary ' under the National Parks and Wildlife Act 1972 by the Government of South Australia on 26 October 1995.
0 2


In [23]:
print(f"Original Sequence Subword tokens: {combined_sentence_subwords}")
print(f"Maksed Sentence Subword tokens:   {masked_subwords}")
print(f"Entailment label: {entailment_label}\n")

# move to device and unsqueeze to add batch dimension
x = {k: v.clone().unsqueeze(0).to(device) for k, v in input.items()}

# forward pass
model.eval()
with torch.no_grad():   
    MLM_logits, entailment_logits, claim_class_logits = model(x['masked_input'], x['attention_mask'], x['segment_ids'])
    logits = MLM_logits.squeeze(0)


# argmax logits
logits_argmax_idx = logits.argmax(dim=-1).tolist()
logits_argmax_subwords = [tokenizer.int2word[i] for i in logits_argmax_idx]

#print(f"Prediction Subword tokens: {logits_argmax_subwords}\n")

for i in range(len(logits)):
    if x['MLM_label'][0][i] != -100:
        # show top 10 predictions for each masked/replaced token
        topk_idx = torch.topk(logits[i], k=10)
        topk_subwords = [tokenizer.int2word[idx] for idx in topk_idx.indices.tolist()]
        print(f"Actual token --> {combined_sentence_subwords[i]}, Masked/Replaced Token --> {masked_subwords[i]}, Predicted Token Top 10 --> {topk_subwords}")
print("")

# compute argmax predictions for masked/replaced tokens
preds = logits.argmax(dim=-1)
mask = (x['MLM_label'][0] != -100)
masked_preds = preds[mask]

# compute argmax entailment prediction
entailment_pred = entailment_logits.argmax(dim=-1).item()

actual_tokens = [tokenizer.int2word[idx] for idx in x['MLM_label'][0][mask].tolist()] 
predicted_tokens = [tokenizer.int2word[idx] for idx in masked_preds.tolist()]

print(f"Actual tokens: {actual_tokens}")
print(f"Predicted tokens: {predicted_tokens}")
print(f"Entailment Logits --> {entailment_logits.tolist()[0]}, Prediction: {entailment_pred}")



Original Sequence Subword tokens: ['[CLS]', '``', 'do', '##min', '##ance', 'hier', '##arch', '##i', '##es', ',', 'diversity', 'and', 'species', 'rich', '##ne', '##s', '##s', 'of', 'va', '##scu', '##lar', 'plants', 'in', 'an', 'alpine', 'me', '##a', '##d', '##ow', ':', 'contrast', '##ing', 'short', 'and', 'medium', 'term', 'responses', 'to', 'si', '##mu', '##l', '##ated', 'global', 'change', "''", '.', '[SEP]', 'in', '2017', ',', 'they', 'held', 'seven', 'tag', 'team', 'championships', 'from', 'seven', 'different', 'promotion', '##s', 'at', 'the', 'same', 'time', ',', 'including', 'the', 'ro', '##h', 'world', 'tag', 'team', 'championship', '.', 'local', 'elections', 'are', 'to', 'be', 'held', 'for', 'wy', '##r', '##e', 'borough', 'council', 'on', '7', 'may', '2015', ',', 'the', 'same', 'day', 'as', 'the', 'united', 'kingdom', 'general', 'election', ',', '2015', 'and', 'other', 'united', 'kingdom', 'local', 'elections', ',', '2015', '##.', 'it', 'was', 'declared', 'as', 'sanctuary', "'",